In [1]:
# Imports
from hmm import HMM, State

### Initial implementation of Hidden Markov Model

In [2]:
# Fixed transitions for example
t_pb = {}
t_pb['T'] = {}
t_pb['F'] = {}
# to state X from state Y
t_pb['T']['T'] = 0.7
t_pb['T']['F'] = 0.3
t_pb['F']['T'] = 0.3 # 1 - self.t_pb['T']['T']
t_pb['F']['F'] = 0.7 # 1 - self.t_pb['T']['F']

In [3]:
h = HMM(t_pb)
print(h.t_pb)

state = {"T": 1, "F": 0}
print(state)
# find converging distribution here
for _ in range(20):
    state = h.transition(state)
    print(state)


{'T': {'T': 0.7, 'F': 0.3}, 'F': {'T': 0.3, 'F': 0.7}}
{'T': 1, 'F': 0}
{'T': 0.7, 'F': 0.3}
{'T': 0.58, 'F': 0.42}
{'T': 0.532, 'F': 0.46799999999999997}
{'T': 0.5128, 'F': 0.48719999999999997}
{'T': 0.50512, 'F': 0.49488}
{'T': 0.502048, 'F': 0.49795199999999995}
{'T': 0.5008192, 'F': 0.4991808}
{'T': 0.50032768, 'F': 0.49967231999999995}
{'T': 0.5001310720000001, 'F': 0.49986892799999993}
{'T': 0.5000524288, 'F': 0.4999475712}
{'T': 0.50002097152, 'F': 0.49997902848}
{'T': 0.500008388608, 'F': 0.499991611392}
{'T': 0.5000033554432, 'F': 0.4999966445568}
{'T': 0.50000134217728, 'F': 0.4999986578227199}
{'T': 0.500000536870912, 'F': 0.4999994631290879}
{'T': 0.5000002147483646, 'F': 0.49999978525163513}
{'T': 0.5000000858993457, 'F': 0.4999999141006539}
{'T': 0.5000000343597382, 'F': 0.4999999656402615}
{'T': 0.5000000137438951, 'F': 0.49999998625610453}
{'T': 0.500000005497558, 'F': 0.4999999945024417}


### New Approach with State class

In [4]:
# Generate states
true = State("T")
false = State("F")

# Link states -> to self, from state dict
true.add({true: 0.7, false: 0.3})
false.add({true: 0.3, false: 0.7})

# Observed states
um_t = State("Ut")
um_f = State("Uf")

# Likelihoods depending on Rt
um_t.add({true: 0.9, false: 0.2})
um_f.add({true: 0.1, false: 0.8})

print(true)
print(false)


T : [('T', 0.7), ('F', 0.3)]
F : [('T', 0.3), ('F', 0.7)]


In [5]:
def rough_equal(a: float, b: float, dec: int) -> bool:
    """Find if two numbers are equal within a decimal point range `dec`."""
    return round(a, dec) == round(b, dec)

# Ensure linking is correct -- account for python inaccuracies with floats
rough_equal(true.given(false), 1-false.given(false), 3)


True

In [6]:
def dist_print(state: dict[State: float]):
    """Short hand function to simplify printing a state distribution."""
    print([(x.id, round(p, 3)) for x,p in state.items()])

# Prior distribution
dist = {true: 1, false: 0}
dist_print(dist)

# Find converging distribution here
for _ in range(20):
    new = {}
    for state in dist.keys():
        new[state] = state.total(dist)
    dist = new
    dist_print(dist)


[('T', 1), ('F', 0)]
[('T', 0.7), ('F', 0.3)]
[('T', 0.58), ('F', 0.42)]
[('T', 0.532), ('F', 0.468)]
[('T', 0.513), ('F', 0.487)]
[('T', 0.505), ('F', 0.495)]
[('T', 0.502), ('F', 0.498)]
[('T', 0.501), ('F', 0.499)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]
[('T', 0.5), ('F', 0.5)]


In [7]:
def iterate_total(dist: dict[State, float], n: int) -> dict[State, float]:
    """Steps through a state distribution from state `start`, `n` times."""
    for _ in range(n):
        dist = {s:s.total(dist) for s in dist.keys()}
    return dist

# Initial distribution - with enough steps, no effect
dist = {true: 1, false: 0}
dist_print(iterate_total(dist, 10))


[('T', 0.5), ('F', 0.5)]


In [8]:
def posterior_total(state: dict[State, float], evidence: dict[State, float], 
                    n: int) -> dict[State, float]:
    """Steps through a state distribution from state `start`, `n` times. Steps 
    consider how observed data `evidence` affects possibilities."""
    for _ in range(n):
        # todo - no handling of None here, need to assert state in `evidence`.
        partials = {s:s.posterior_x_evidence(state, evidence.likelihood) for s in state.keys()}
        # Find the normalising factor
        alpha = 1/sum(partials.values())
        state = {s:v*alpha for s,v in partials.items()}
    return state

# Initial distribution - with enough steps, no effect
state = {true: 1, false: 0}
dist_print(posterior_total(state, um_t, 10))


[('T', 0.897), ('F', 0.103)]
